In [1]:
import numpy as np

In [2]:
np.__file__, np.__version__

('/Users/kunal21/opt/miniconda3/envs/gnnnlp2/lib/python3.10/site-packages/numpy/__init__.py',
 '1.24.0')

#### TF TEST

In [3]:
import tensorflow as tf

In [4]:
tf.reduce_sum(tf.random.normal([1000, 1000]))

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



<tf.Tensor: shape=(), dtype=float32, numpy=-640.41174>

In [5]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
import torch
torch.__version__

'2.0.1'

In [7]:
import transformers as tr
tr.__version__ 

/Users/kunal21/opt/miniconda3/envs/gnnnlp2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'4.28.1'

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import os
import re

##### 
- Downloading a text dataset
- Visualizing text data
- Converting text into numbers using tokenization
- Turning our tokenized text into an embedding
- Modelling a text dataset
- Starting with a baseline (TF-IDF)
- Building several deep learning text models
- Dense, LSTM, GRU, Conv1D, Transfer learning
- Comparing the performance of each our models
- Combining our models into an ensemble
- Saving and loading a trained model
- Find the most wrong predictions

### Data Pre-Processing

In [28]:
book =[]
with open('Jonathan_Livingston_Seagull.txt', mode='r') as file:
    book=file.readlines()

book    

['Jonathan Livingston Seagull - a story \n',
 '\n',
 '\n',
 '\n',
 'PART ONE\n',
 '\n',
 'it was morning, and the new sun sparkled gold across the ripples of a gentle \n',
 'sea. \n',
 '\n',
 'A mile from shore a fishing boat chummed the water, and the word for \n',
 'Breakfast Flock flashed through the air, till a crowd of a thousand seagulls \n',
 'came to dodge and fight for bits of food. It was another busy day beginning. \n',
 '\n',
 'But way off alone, out by himself beyond boat and shore, Jonathan \n',
 'Livingston Seagull was practicing. A hundred feet in the sky he lowered his \n',
 'webbed feet, lifted his beak, and strained to hold a painful hard twisted curve \n',
 'through his wings. The curve meant that he would fly slowly, and now he \n',
 'slowed until the wind was a whisper in his face, until the ocean stood still \n',
 'beneath him. He narrowed his eyes in fierce concentration, held his breath, \n',
 'forced one ... single ... more ... inch ... of ... curve .... Then 

In [29]:
len(book)

1175

#### re-formatting sentences 

In [30]:
sentence_list = ['\n']

for x in range(0,len(book)):    
    if (book[x] =='\n'):
        sentence_list.append(book[x])
    else:
        if(sentence_list[-1]!='\n'):
            sentence_list[-1] = (sentence_list[-1]+' '+ book[x]).replace('\n','')
        else:
            sentence_list.append(book[x].replace('\n',''))
        
sentence_list   

['\n',
 'Jonathan Livingston Seagull - a story ',
 '\n',
 '\n',
 '\n',
 'PART ONE',
 '\n',
 'it was morning, and the new sun sparkled gold across the ripples of a gentle  sea. ',
 '\n',
 'A mile from shore a fishing boat chummed the water, and the word for  Breakfast Flock flashed through the air, till a crowd of a thousand seagulls  came to dodge and fight for bits of food. It was another busy day beginning. ',
 '\n',
 'But way off alone, out by himself beyond boat and shore, Jonathan  Livingston Seagull was practicing. A hundred feet in the sky he lowered his  webbed feet, lifted his beak, and strained to hold a painful hard twisted curve  through his wings. The curve meant that he would fly slowly, and now he  slowed until the wind was a whisper in his face, until the ocean stood still  beneath him. He narrowed his eyes in fierce concentration, held his breath,  forced one ... single ... more ... inch ... of ... curve .... Then his feathers  ruffled, he stalled and fell. ',
 '\n',
 

In [31]:
len(sentence_list)

687

In [32]:
1175 - 687 #no of sentences for summarization after formatting - also contains \n - to be removed further

488

In [33]:
def get_words(input_sentences_list):
    words_list =[]

    [words_list.append(len(x)) for x in input_sentences_list]
    words_list.sort()

    print('sentenaces - min length : '+ str(words_list[0]) + ' max length : '+ str(words_list[-1]), ' mean - ', np.mean(words_list))

get_words(sentence_list)

sentenaces - min length : 1 max length : 927  mean -  72.58806404657933


#### Extrating Chapters

In [37]:
chapters = [y[0] for y in [re.findall(r"^PART.*|^\d*\s\..*",x) for x in sentence_list] if len(y)>0 and len(y[0])>1]
chapters

['PART ONE', 'PART TWO', 'PART THREE']

In [38]:
sentence_list[0:sentence_list.index('PART ONE')]

['\n', 'Jonathan Livingston Seagull - a story ', '\n', '\n', '\n']

In [39]:
len(chapters)

3

#### creating book dataset

In [40]:
book_dict = {}

start_index = 0
for b in range(0,len(chapters)):
    if b < len(chapters)-1:           
        book_dict[chapters[b]] = sentence_list[start_index:sentence_list.index(chapters[b+1])]
        start_index = sentence_list.index(chapters[b+1])+1
    else:
        book_dict[chapters[b]] = sentence_list[start_index:]
    
    
    
book_dict   

{'PART ONE': ['\n',
  'Jonathan Livingston Seagull - a story ',
  '\n',
  '\n',
  '\n',
  'PART ONE',
  '\n',
  'it was morning, and the new sun sparkled gold across the ripples of a gentle  sea. ',
  '\n',
  'A mile from shore a fishing boat chummed the water, and the word for  Breakfast Flock flashed through the air, till a crowd of a thousand seagulls  came to dodge and fight for bits of food. It was another busy day beginning. ',
  '\n',
  'But way off alone, out by himself beyond boat and shore, Jonathan  Livingston Seagull was practicing. A hundred feet in the sky he lowered his  webbed feet, lifted his beak, and strained to hold a painful hard twisted curve  through his wings. The curve meant that he would fly slowly, and now he  slowed until the wind was a whisper in his face, until the ocean stood still  beneath him. He narrowed his eyes in fierce concentration, held his breath,  forced one ... single ... more ... inch ... of ... curve .... Then his feathers  ruffled, he stall

In [41]:
book_dict.keys()

dict_keys(['PART ONE', 'PART TWO', 'PART THREE'])

In [42]:
book_dict.get('PART ONE')

['\n',
 'Jonathan Livingston Seagull - a story ',
 '\n',
 '\n',
 '\n',
 'PART ONE',
 '\n',
 'it was morning, and the new sun sparkled gold across the ripples of a gentle  sea. ',
 '\n',
 'A mile from shore a fishing boat chummed the water, and the word for  Breakfast Flock flashed through the air, till a crowd of a thousand seagulls  came to dodge and fight for bits of food. It was another busy day beginning. ',
 '\n',
 'But way off alone, out by himself beyond boat and shore, Jonathan  Livingston Seagull was practicing. A hundred feet in the sky he lowered his  webbed feet, lifted his beak, and strained to hold a painful hard twisted curve  through his wings. The curve meant that he would fly slowly, and now he  slowed until the wind was a whisper in his face, until the ocean stood still  beneath him. He narrowed his eyes in fierce concentration, held his breath,  forced one ... single ... more ... inch ... of ... curve .... Then his feathers  ruffled, he stalled and fell. ',
 '\n',
 

In [43]:
part_three = book_dict['PART THREE']
part_three

['\n',
 'Jonathan circled slowly over the Far Cliffs, watching. This rough young  Fletcher Gull ws very nearly a perfect flight-student. He was strong and light  and quick in the air, but far and away more important, he had a blazing drive  to learn to fly. ',
 '\n',
 'Here he came this minute, a blurred grey shape roaring out of a dive,  flashing one hundred fifty miles per hour past his instructor. He pulled  abruptly into another try at a sixteen-pint vertical slow roll, calling the points  out loud. ',
 '\n',
 '\n',
 '\n',
 "’...eight ...nine ...ten ...see-Jonthan-I'm-running-out-of-airspeed ...eleven ",
 '\n',
 '\n',
 '\n',
 '\n',
 '. ..I-want-good-sharp-stops-like-yours . . . twelve . . . but-blast-it-I-just-can \'t-make  ...thirteen ...these-last-three-points ...without ...fourteen ...aaakkk!" ',
 '\n',
 "Fletcher's whipstall at the top was all the worse for his rage and fury at  failing. He fell backward, tumbled, slammed savagely into and inverted spin,  and recovered at last,

In [44]:
#'NO OF SENTENCES PER SECTION OF BOOK = '
[ x +' : '+ str(len(book_dict.get(x))) for x in book_dict.keys()]

['PART ONE : 215', 'PART TWO : 224', 'PART THREE : 246']

#### EXTRATING PARAGRAPHS FOR SUMMARIZATION

 -  As observed from data, there is 3 lines of gap between paragraphs, so extracting paragraph list for summarization


In [45]:
book_dict_paragraphs ={}

for x in book_dict.keys():
    paragraph_list=[]
    next_line_counter = 0
    for line in book_dict.get(x):            
        if line != '\n':
            if next_line_counter==3:
                paragraph_list.append(line) 
            else:                
                if len(paragraph_list)!=0:
                        paragraph_list[-1]=paragraph_list[-1]+' '+line
                else:
                    paragraph_list.append(line)
            next_line_counter=0
        else:                
            next_line_counter+=1


    book_dict_paragraphs[x] = paragraph_list
    paragraph_list=[]

            
book_dict_paragraphs.get('PART ONE')[0]

'Jonathan Livingston Seagull - a story '

In [46]:
book_dict_paragraphs.keys()

dict_keys(['PART ONE', 'PART TWO', 'PART THREE'])

In [47]:
print('comparing sentences list with paragraphs after formatting')

for x in book_dict.keys():
    print(x, 'book_dict :: ',len(book_dict.get(x)) ,'book_dict_paragraphs :: ',len(book_dict_paragraphs.get(x)))

comparing sentences list with paragraphs after formatting
PART ONE book_dict ::  215 book_dict_paragraphs ::  11
PART TWO book_dict ::  224 book_dict_paragraphs ::  11
PART THREE book_dict ::  246 book_dict_paragraphs ::  12


##### Finding out average length of words for each sentence for tokenization

In [48]:
[ get_words(book_dict_paragraphs.get(keys)) for keys in book_dict_paragraphs.keys()]

sentenaces - min length : 38 max length : 4473  mean -  1525.090909090909
sentenaces - min length : 89 max length : 3698  mean -  1506.090909090909
sentenaces - min length : 142 max length : 3673  mean -  1361.5


[None, None, None]

In [49]:

for keys in book_dict_paragraphs.keys():
    print('\n'+keys)
    get_words(book_dict_paragraphs.get(keys))
    print('\n')
    print(max(book_dict_paragraphs.get(keys), key=len))



PART ONE
sentenaces - min length : 38 max length : 4473  mean -  1525.090909090909


PART ONE it was morning, and the new sun sparkled gold across the ripples of a gentle  sea.  A mile from shore a fishing boat chummed the water, and the word for  Breakfast Flock flashed through the air, till a crowd of a thousand seagulls  came to dodge and fight for bits of food. It was another busy day beginning.  But way off alone, out by himself beyond boat and shore, Jonathan  Livingston Seagull was practicing. A hundred feet in the sky he lowered his  webbed feet, lifted his beak, and strained to hold a painful hard twisted curve  through his wings. The curve meant that he would fly slowly, and now he  slowed until the wind was a whisper in his face, until the ocean stood still  beneath him. He narrowed his eyes in fierce concentration, held his breath,  forced one ... single ... more ... inch ... of ... curve .... Then his feathers  ruffled, he stalled and fell.  Seagulls, as you know, never f

In [50]:
len(book_dict_paragraphs.get('PART ONE')[0])

38

In [51]:
MAX_LENGTH = 2048
MIN_LENGTH = 32

### USING TRASNSFORMERS FOR SUMMARIZATION

#### USING PEGASUS

In [52]:
from transformers import pipeline

In [70]:
sample_text = book_dict_paragraphs.get('PART ONE')[6]
len(sample_text)

559

In [72]:
summarizer = pipeline(task='summarization', model='google/pegasus-xsum')
summarizer(sample_text)


[{'summary_text': 'In our series of letters from African journalists, film-maker and columnist Farai Sevenzo reflects on the importance of learning to fly.'}]

#### USING BART

In [73]:
summarizer = pipeline("summarization", model = "facebook/bart-large-cnn")
summarizer(sample_text)

Your max_length is set to 142, but you input_length is only 138. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=69)


[{'summary_text': "When Jonathan Seagull joined the Flock on the beach, it was full night. He was dizzy and terribly tired. Yet in delight he flew a loop to landing, with a snap roll just before touchdown. When they hear of it, he thought, of the  Breakthrough, they'll be wild with joy. How much more there is now to  living!"}]

#### USING T5

In [74]:
summarizer = pipeline("summarization", model= "csebuetnlp/mT5_multilingual_XLSum" )
summarizer(sample_text)

/Users/kunal21/opt/miniconda3/envs/gnnnlp2/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


[{'summary_text': 'Jonathan Seagull has spent more than a decade fishing on the Isle of Wight. He was inspired by the Flock when he went to the beach.'}]

#### USING AutoTokenizer and AutoModel

In [75]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [80]:
sample_text = book_dict_paragraphs.get('PART ONE')[3]
len(sample_text)

2464

In [81]:
###USING PEGAGUS

model = AutoModelForSeq2SeqLM.from_pretrained('google/pegasus-xsum')
tokenizer = AutoTokenizer.from_pretrained('google/pegasus-xsum')
tokens_input = tokenizer.encode("summarize:"+ sample_text,return_tensors='pt', 
                                max_length=512, truncation=True)
ids = model.generate(tokens_input, min_length=80,max_length=120)
summary_pegasus = tokenizer.decode(ids[0], skip_special_tokens=True)

summary_pegasus

'From two thousand feet he tried again, rolling into his dive, beak straight down, wings full out and stable from the moment he passed fifty miles per hour. Jonathan Seagull tried again, rolling into his dive, beak straight down, wings full out and stable from the moment he passed fifty miles per hour. He pushed wearily away from the dark water and flew toward the land, grateful for what he had learned about work-saving low-altitude flying.'

In [82]:
#USING BART
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')

tokens_input = tokenizer.encode("summarize:"+ sample_text,return_tensors='pt', 
                                max_length=512, truncation=True)
ids = model.generate(tokens_input, min_length=80,max_length=120)
summary_bart = tokenizer.decode(ids[0], skip_special_tokens=True)

summary_bart

'Jonathan Seagull is a seagull who learned to fly by accident. He crashed into the sea at a speed of ninety miles per hour. He vowed to become a normal gull and fly home to the flock. The story is about his journey back to the sea after he learned how to fly. The book is published by Simon & Schuster at $24.99. For more information on the book, go to www.simonandschuster.com.'

In [83]:
#USING T5
model = AutoModelForSeq2SeqLM.from_pretrained('csebuetnlp/mT5_multilingual_XLSum')
tokenizer = AutoTokenizer.from_pretrained('csebuetnlp/mT5_multilingual_XLSum')

tokens_input = tokenizer.encode("summarize:"+ sample_text,return_tensors='pt', 
                                max_length=512, truncation=True)
ids = model.generate(tokens_input, min_length=80,max_length=120)
summary_t5 = tokenizer.decode(ids[0], skip_special_tokens=True)

summary_t5

/Users/kunal21/opt/miniconda3/envs/gnnnlp2/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


"Jonathan Seagull has become the world's fastest seagull. He is a relatively low-altitude flying enthusiast, but he doesn't believe he will be able to survive. The BBC spoke to him about the challenges he has faced, and why he is not allowed to escape."

#### evaluating using ROUGE scores



In [84]:
from rouge import Rouge

In [85]:
ROUGE = Rouge()

In [86]:
ROUGE.get_scores(sample_text, summary_pegasus)

[{'rouge-1': {'r': 1.0, 'p': 0.1865079365079365, 'f': 0.3143812682533753},
  'rouge-2': {'r': 0.9615384615384616,
   'p': 0.11933174224343675,
   'f': 0.21231422308878886},
  'rouge-l': {'r': 1.0, 'p': 0.1865079365079365, 'f': 0.3143812682533753}}]

In [87]:
ROUGE.get_scores(sample_text, summary_bart)

[{'rouge-1': {'r': 0.5818181818181818,
   'p': 0.12698412698412698,
   'f': 0.20846905243344757},
  'rouge-2': {'r': 0.16901408450704225,
   'p': 0.028639618138424822,
   'f': 0.04897958935868401},
  'rouge-l': {'r': 0.5454545454545454,
   'p': 0.11904761904761904,
   'f': 0.19543973647253554}}]

In [88]:
ROUGE.get_scores(sample_text, summary_t5)

[{'rouge-1': {'r': 0.5555555555555556,
   'p': 0.07936507936507936,
   'f': 0.1388888867013889},
  'rouge-2': {'r': 0.06976744186046512,
   'p': 0.007159904534606206,
   'f': 0.012987011298795223},
  'rouge-l': {'r': 0.5277777777777778,
   'p': 0.07539682539682539,
   'f': 0.1319444422569445}}]

###### CONCLUSION - looking at the scores, BART seems to be a good choice for summarizaiton with hightest rouge score

### USING BART for generating summaries

In [89]:
from tqdm import tqdm

In [90]:
book_dict_summary ={}

model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')

for x in tqdm(book_dict_paragraphs.keys()):
    
    summary_list = []
    for lines in tqdm(book_dict_paragraphs.get(x)):
        tokens_input = tokenizer.encode("summarize:"+ lines,return_tensors='pt',max_length=512, truncation=True)
        ids = model.generate(tokens_input,min_length=MIN_LENGTH, max_length=MAX_LENGTH)
        summary_bart = tokenizer.decode(ids[0], skip_special_tokens=True)
        summary_list.append([lines, summary_bart])

    book_dict_summary[x] = summary_list
        
book_dict_summary.get('PART ONE')[0]    
# for x in book_dict_paragraphs.keys():
#     summary_list=[]
#     for lines in book_dict_paragraphs.get(x):         
        
#         tokens_input = tokenizer.encode("summarize:"+ lines,return_tensors='pt', 
#                                         max_length=512, truncation=True)
#         ids = model.generate(tokens_input, max_length=80)
#         summary_bart = tokenizer.decode(ids[0], skip_special_tokens=True)

#     summary_bart\

100%|██████████| 3/3 [02:43<00:00, 54.43s/it]


['Jonathan Livingston Seagull - a story ',
 'summarize:Jonathan Livingston Seagull - a story  about a seagull. Jonathan Livingston Segull -  a story about a man who lost his life in a bird crash.']

In [91]:
book_dict_summary.keys()

dict_keys(['PART ONE', 'PART TWO', 'PART THREE'])

In [92]:
for x in (book_dict_summary.get('PART TWO')):
    print(x[0])
    print('summary ->>' ,x[1])
    print('\n')

So this is heaven, he thought, and he had to smile at himself. It was hardly  respectful to analyse heaven in the very moment that one flies up to enter it.  As he came from Earth now, above the clouds and in close formation with  the two brilliant gulls, he saw that his own body was growing as bright as  theirs. True, the same young Jonathan Seagull was there that has always lived  behind his golden eyes, but the outer form had changed.  It felt like a seagull body, but already it flew far better than his old one had  ever flown. Why, with half the effort, he though, I'll get twice the speed,  twice the performance of my best days on earth!  His feathers glowed brilliant white now, and his wings were smooth and  perfect as sheets of polished silver. He began, delightedly, to learn about  them, to press power into these new wings.  At two hundred fifty miles per hour he felt that he was nearing his level-  flight maximum speed. At two hundred seventy-three he thought that he was  flyin

In [93]:
import json

In [94]:
book_summary_json = json.dumps(book_dict_summary)
book_summary_json

'{"PART ONE": [["Jonathan Livingston Seagull - a story ", "summarize:Jonathan Livingston Seagull - a story  about a seagull. Jonathan Livingston Segull -  a story about a man who lost his life in a bird crash."], ["PART ONE it was morning, and the new sun sparkled gold across the ripples of a gentle  sea.  A mile from shore a fishing boat chummed the water, and the word for  Breakfast Flock flashed through the air, till a crowd of a thousand seagulls  came to dodge and fight for bits of food. It was another busy day beginning.  But way off alone, out by himself beyond boat and shore, Jonathan  Livingston Seagull was practicing. A hundred feet in the sky he lowered his  webbed feet, lifted his beak, and strained to hold a painful hard twisted curve  through his wings. The curve meant that he would fly slowly, and now he  slowed until the wind was a whisper in his face, until the ocean stood still  beneath him. He narrowed his eyes in fierce concentration, held his breath,  forced one ..

In [95]:
with open('book_dict_summary.json','w') as outfile:
    outfile.write(book_summary_json)

###### NEXT - GNN_Phase_II doc